<a href="https://colab.research.google.com/github/amaithi-sam/LLM-Alignment/blob/main/LLM_finetuning_LoRa_%26_QLoRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installing required frameworks

In [1]:
!pip install transformers trl peft accelerate datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
 

In [1]:
from huggingface_hub import notebook_login
notebook_login()

# Instruction Finetuning - SFT

In [9]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from accelerate import Accelerator
import os


In [10]:
sft_config = {
    "model_ckpt": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "load_in_4bit": True,
    "device_map" : {"": Accelerator().local_process_index},
    "torch_dtype" : torch.float16,
    "trust_remote_code" : True,
    "use_lora": True,
    "r": 16,
    "lora_alpha" : 16,
    "lora_dropout" : 0.05,
    "bias" : "none",
    "task_type": "CASUAL_LM",
    "target_modules" : ["q_proj", "v_proj"],
    "output_dir" : "sft-tinyLlama-chatbot",
    "per_device_train_batch_size": 1,
    "gradient_accumulation_steps" : 1,
    "optim" : "paged_adamw_32bit",
    "learning_rate" : 2e-4,
    "lr_scheduler_type" : "cosine",
    "save_strategy" : "epoch",
    "logging_steps" : 100,
    "num_train_epochs" : 1,
    "max_steps" : 250,
    "fp16" : True,
    "push_to_hub" : True,
    "train_cln_name" : "text",
    "packing" : False,
    "max_seq_length" : 512,
    "neftune_noise_alpha": 5
}

In [11]:
class TrainSFT:

  def __init__(self, data, config):
    self.data = data
    self.config = config

  def prepare_lora_model(self):

    self.lora_config = LoraConfig(
        r = self.config['r'],
        lora_alpha = self.config['lora_alpha'],
        lora_dropout = self.config['lora_dropout'],
        bias = self.config['bias'],
        task_type = self.config['task_type'],
        target_modules = self.config['target_modules']
    )

    self.model = get_peft_model(self.model, self.lora_config)

  def load_model_tokenizer(self):

    self.model = AutoModelForCausalLM.from_pretrained(
        self.config['model_ckpt'],
        load_in_4bit = self.config['load_in_4bit'],
        device_map = self.config['device_map'],
        torch_dtype = self.config['torch_dtype']
    )
    self.model.config.use_cache = False
    self.model.config.pretraining_tp = 1
    self.model = prepare_model_for_kbit_training(self.model)

    if self.config['use_lora']:
      self.prepare_lora_model()

    self.tokenizer = AutoTokenizer.from_pretrained(self.config['model_ckpt'])
    self.tokeniezr_pad = self.tokenizer.eos_token

  def set_training_args(self):

    return TrainingArguments(
        output_dir = self.config['output_dir'],
        per_device_train_batch_size = self.config['per_device_train_batch_size'],
        gradient_accumulation_steps = self.config['gradient_accumulation_steps'],
        optim = self.config['optim'],
        learning_rate = self.config['learning_rate'],
        lr_scheduler_type = self.config['lr_scheduler_type'],
        save_strategy = self.config['save_strategy'],
        logging_steps = self.config['logging_steps'],
        num_train_epochs= self.config['num_train_epochs'],
        # max_steps = self.config['max_steps'],
        fp16 = self.config['fp16'],
        push_to_hub = self.config['push_to_hub'],
        neftune_noise_alpha = self.config['neftune_noise_alpha']
          )

  def create_trainer(self):

    self.load_model_tokenizer()
    if self.config['use_lora']:
      print(self.model.print_trainable_parameters())

      self.trainer = SFTTrainer(
          model = self.model,
          train_dataset = self.data,
          peft_config = self.lora_config,
          dataset_text_field = self.config['train_cln_name'],
          args = self.set_training_args(),
          tokenizer = self.tokenizer,
          packing = self.config['packing'],
          max_seq_length = self.config['max_seq_length']
      )

    else:

      self.trainer = SFTTrainer(
            model = self.model,
            train_dataset = self.data,
            dataset_text_field = self.config['train_cln_name'],
            args = self.set_training_args(),
            tokenizer = self.tokenizer,
            packing = self.config['packing'],
            max_seq_length = self.config['max_seq_length']
      )
  def train_and_save_model(self):
    self.create_trainer()
    self.trainer.train()
    self.trainer.save_model(self.config['output_dir'])
    self.tokenizer.save_pretrained(self.config['output_dir'])

In [12]:
def create_data():
    data = load_dataset("tatsu-lab/alpaca", split="train")
    data_df = data.to_pandas()
    data_df = data_df[:700]
    data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    data = Dataset.from_pandas(data_df)
    return data

data = create_data()

In [13]:
data

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 700
})

In [14]:
train_sft = TrainSFT(data, sft_config)

In [15]:
train_sft.train_and_save_model()

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

...

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
None


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need

Step,Training Loss
100,2.187500
200,1.708400
300,1.658000
400,1.551100
500,1.541900
600,1.554800
700,1.560800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo